In [84]:
import pandas as pd
import seaborn as sns
import csv
import sqlite3
import matplotlib.pyplot as plt

In [85]:
df = pd.read_csv('owid-covid-data.csv')

In [86]:
df.to_sql('covid', sqlite3.connect("covid.db"), if_exists='append', index = False)

In [87]:
conn = sqlite3.connect('covid.db')

In [88]:
# Number of tables

df = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
df

,name
0,covid


In [89]:
# Name of columns

df = pd.read_sql_query("SELECT sql FROM sqlite_master WHERE tbl_name = 'covid' AND type = 'table';", conn)
df

,sql
0,"CREATE TABLE ""covid"" (\n""iso_code"" TEXT,\n ""c..."


In [90]:
# 1. How many countries are included in our data set?

df = pd.read_sql_query("SELECT COUNT(DISTINCT location) FROM covid;", conn)
df

,COUNT(DISTINCT location)
0,219


In [91]:
 # 2. When was the most recent entry?
    
df = pd.read_sql_query("SELECT date FROM covid ORDER BY date DESC LIMIT 1;", conn)
df

,date
0,2021-05-06


In [92]:
df = pd.read_sql_query("SELECT MAX (date) FROM covid;", conn)
df

,MAX (date)
0,2021-05-06


In [93]:
# 3. What country had the highest single day deaths per million? 

df = pd.read_sql_query("SELECT MAX(new_deaths_per_million), location FROM covid;", conn)
df

,MAX(new_deaths_per_million),location
0,218.329,Ecuador


In [94]:
#  4. Select any country. How many times have the deaths per million exceeded 1,000 in one day? 

df = pd.read_sql_query("SELECT COUNT(*) from covid WHERE location = 'France' and new_deaths > 1000;", conn)
df

,COUNT(*)
0,60


In [103]:
# 5. Extract the deaths per million for US, France, and India. Plot the time series data and compare 
# results to what the media is currently saying

df2 = pd.read_sql_query("SELECT DISTINCT date, new_deaths_per_million, location FROM covid WHERE location IN ('France', 'India', 'United States') ORDER BY date ASC;", conn)
df2

,date,new_deaths_per_million,location
0,2020-01-22,NaN,United States
1,2020-01-23,NaN,United States
2,2020-01-24,NaN,France
3,2020-01-24,NaN,United States
4,2020-01-25,NaN,France
...,...,...,...
1398,2021-05-05,2.884,India
1399,2021-05-05,2.344,United States
1400,2021-05-06,3.214,France
1401,2021-05-06,0.000,India


In [106]:
df3 = pd.DataFrame(df2)
df3['date'] = df3['date'].astype(dtype='datetime64[ns]')
df3

,date,new_deaths_per_million,location
0,2020-01-22,NaN,United States
1,2020-01-23,NaN,United States
2,2020-01-24,NaN,France
3,2020-01-24,NaN,United States
4,2020-01-25,NaN,France
...,...,...,...
1398,2021-05-05,2.884,India
1399,2021-05-05,2.344,United States
1400,2021-05-06,3.214,France
1401,2021-05-06,0.000,India


In [ ]:
fig, ax = plt.subplots(figsize=(26,10))
sns.set_context("paper", font_scale=2.5, rc={"lines.linewidth": 3.5})
plt.rcParams['figure.dpi'] = 360
sns.lineplot(data=df3, x="date", y="new_deaths_per_million", hue = 'location')
plt.xlabel("date") 
plt.ylabel("new_deaths_per_million") 
plt.title("Evolution of covid deaths per million")
horizontalalignment='right'

In [ ]:
# 6. Plot the deaths per million and cases per million by continent for your birthday in 2020. What does the data say?


df4 = pd.read_sql_query("SELECT new_deaths_per_million, new_cases_per_million, continent, date FROM covid WHERE date = '2020-08-28' GROUP BY continent;", conn)
df4

In [ ]:
df2.isnull().sum()

In [ ]:
sns.set_theme(style="whitegrid")
sns.barplot(x="continent", y="new_cases_per_million", data=df4)